In [ ]:
using Serialization
using Statistics
using StatsBase
using StatsPlots
using CircularArrays
using CSV, DataFrames
using KernelDensity
using Distributions
using Random

In [ ]:
Atgff = CSV.File("AP000423.gff"; comment = "#", header = ["accession", "software", "feature", "start", "stop", "score", "strand", "phase", "attributes"]) |> DataFrame
genome_length = first(Atgff[Atgff.feature .== "region", :stop])

function rc(x::Real)
    genome_length - x + 1
end

structuralRNA = sort!(filter(x->x.feature ∈ ["tRNA", "rRNA"], Atgff), :start)

function filteroutstructuralRNA!(data, strand)
    mask = filter(x->x.strand == strand, structuralRNA)
    for rna in eachrow(mask)
        if strand == "+"
            data[rna.start:rna.stop] .= 0
        else
            data[rc(rna.stop):rc(rna.start)] .= 0
        end
    end
    data
end


In [ ]:
footprints_gff = CSV.File("Known_short_AP000423.gff"; comment = "#", header = ["accession", "software", "feature", "start", "stop", "score", "strand", "phase", "attributes", "present"]) |> DataFrame
filter!(x->occursin("present", x.present) , footprints_gff)
select!(footprints_gff, Not(:present))

In [ ]:
window = 50
N = 10000

function log_normalise(fwd_data, rev_data)
    log_fwd_data = log.(fwd_data .+ 1)
    log_rev_data = log.(rev_data .+ 1)
    log_fwd_data .-= min(minimum(log_fwd_data), minimum(log_rev_data))
    log_fwd_data ./= max(maximum(log_fwd_data), maximum(log_rev_data))
    log_rev_data .-= min(minimum(log_fwd_data), minimum(log_rev_data))
    log_rev_data ./= max(maximum(log_fwd_data), maximum(log_rev_data))
    log_fwd_data, log_rev_data
end

function sample_from_kde(data::Vector{Int}, N::Int; boundaries = (1,50))
    h = KernelDensity.default_bandwidth(data)
    sampled = Vector{Int}(undef, N)
    i = 1
    while i <= N
        sample = round(Int, rand(Normal(rand(data), h)))
        if first(boundaries) <= sample <= last(boundaries)
            sampled[i] = sample
            i += 1
        end
    end
    sampled
end

function sample_from_kde(data::Vector{Float64}, N::Int; boundaries = (0.0,1.0))
    h = KernelDensity.default_bandwidth(data)
    sampled = Vector{Float64}(undef, N)
    i = 1
    while i <= N
        sample = rand(Normal(rand(data), h))
        if first(boundaries) <= sample <= last(boundaries)
            sampled[i] = sample
            i += 1
        end
    end
    sampled
end


In [ ]:
footprint_lengths = footprints_gff.stop .- footprints_gff.start .+ 1
kde_lengths = kde(footprint_lengths, boundary=(1,window))
plot(kde_lengths; color = :dodgerblue, label = "observed footprint_lengths")
simulated_lengths = sample_from_kde(footprint_lengths, N; boundaries = (15,40))
plot!(kde(simulated_lengths); color = :skyblue, label = "simulated footprint_lengths")


In [ ]:
fwd_conservation = CircularVector(deserialize("arabidopsis_conservation.bin"))
rev_conservation = reverse(fwd_conservation)
fwd_conservation, rev_conservation = log_normalise(fwd_conservation, rev_conservation)

conservation_footprints = Vector{Vector{Float64}}()
for f in eachrow(footprints_gff)
    midpoint = f.start + (f.stop - f.start)/2
    if mod(f.stop - f.start, 2) == 0; midpoint += 0.5; end
    frange = Int(midpoint - 24.5):Int(midpoint + 24.5)
    push!(conservation_footprints, fwd_conservation[frange])
end

conservation_peak_std = mean(std.(getindex.(conservation_footprints, Ref(13:37))))
conservation_flanks_std = (mean(std.(getindex.(conservation_footprints, Ref(1:12)))) + mean(std.(getindex.(conservation_footprints, Ref(38:50)))))/2.0

In [ ]:
peaks = mean.(getindex.(conservation_footprints, Ref(20:30)))
simulated_conservation_peaks = sample_from_kde(peaks, N)
plot(kde(peaks); label = "conservation peaks", color = :green, xguide = "transformed conservation", yguide = "density")
plot!(kde(simulated_conservation_peaks); label = "simulated conservation peaks", color = :palegreen, xlims = (0,1))

In [ ]:
flanks = (mean.(getindex.(conservation_footprints, Ref(1:5))) .+ mean.(getindex.(conservation_footprints, Ref(46:50)))) ./ 2.0 ./ peaks
simulated_conservation_flanks = sample_from_kde(flanks, N)
plot(kde(flanks); color = :crimson, label = "observed conservation flanks", xguide = "proportion of peak height", yguide = "density")
plot!(kde(simulated_conservation_flanks); color = :tomato, label = "simulated conservation flanks", xlims = (0,1))

In [ ]:
fwd_srna = deserialize("srna_fwd.bin")
rev_srna = deserialize("srna_rev.bin")

filteroutstructuralRNA!(fwd_srna, "+")
filteroutstructuralRNA!(rev_srna, "-")

fwd_srna, rev_srna = log_normalise(fwd_srna, rev_srna)

srna_footprints = Vector{Vector{Float64}}()
for f in eachrow(footprints_gff)
    midpoint = f.start + (f.stop - f.start)/2
    if mod(f.stop - f.start, 2) == 0; midpoint += 0.5; end
    if f.strand == "-"; midpoint = rc(midpoint); end
    frange = Int(midpoint - 24.5):Int(midpoint + 24.5)
    push!(srna_footprints, f.strand == "+" ? fwd_srna[frange] : rev_srna[frange])
end

In [ ]:
plot()
for f in srna_footprints
    plot!(f; color = :gray, alpha = 0.5, label = false, xguide = "nucleotide", yguide = "transformed read depth")
end
plot!()

In [ ]:
srna_peak_std = median(std.(getindex.(srna_footprints, Ref(13:37))))
srna_flanks_std = (median(std.(getindex.(srna_footprints, Ref(1:12)))) + median(std.(getindex.(srna_footprints, Ref(38:50)))))/2.0

In [ ]:
peaks = mean.(getindex.(srna_footprints, Ref(20:30)))
simulated_peaks_srna = sample_from_kde(peaks, N)
plot(kde(peaks); label = "observed sRNA peaks", color = :green, xguide = "transformed read depth", yguide = "density")
plot!(kde(simulated_peaks_srna); color = :palegreen, label = "simulated sRNA peaks", xlims = (0,1))

In [ ]:
flanks = (mean.(getindex.(srna_footprints, Ref(1:5))) .+ mean.(getindex.(srna_footprints, Ref(46:50)))) ./ 2.0 ./ peaks
simulated_descents_srna = sample_from_kde(flanks, N)
plot(kde(flanks); color = :crimson, label = "observed sRNA flank descents", xguide = "proportion of peak height", yguide = "density")
plot!(kde(simulated_descents_srna); color = :tomato, label = "simulated sRNA flank descents", xlims = (0,1))

In [ ]:
fwd_fiveprime = CircularVector(deserialize("ligated_vs_untreated.fwd.five.bin"))
filteroutstructuralRNA!(fwd_fiveprime, "+")
rev_fiveprime = CircularVector(deserialize("ligated_vs_untreated.rev.five.bin"))
filteroutstructuralRNA!(rev_fiveprime, "-")

fwd_fiveprime, rev_fiveprime = log_normalise(fwd_fiveprime, rev_fiveprime)

In [ ]:
fiveprime_termini = filter(x->x>0, vcat(fwd_fiveprime, rev_fiveprime))
plot(kde(fiveprime_termini); label = "observed 5' termini", color = :green, xguide = "transformed read depth", yguide = "density", xlims = (0,1))
simulated_fiveprime_termini = sample_from_kde(fiveprime_termini.data, N)
plot!(kde(simulated_fiveprime_termini); color = :palegreen, label = "simulated 5' termini")

In [ ]:
fwd_threeprime = CircularVector(deserialize("TAP_vs_untreated.fwd.three.bin"))
filteroutstructuralRNA!(fwd_threeprime, "+")
rev_threeprime = CircularVector(deserialize("TAP_vs_untreated.rev.three.bin"))
filteroutstructuralRNA!(rev_threeprime, "-")

fwd_threeprime, rev_threeprime = log_normalise(fwd_threeprime, rev_threeprime)

In [ ]:
threeprime_termini = filter(x->x>0, vcat(fwd_threeprime, rev_threeprime))
plot(kde(threeprime_termini); label = "observed 3' termini", color = :crimson, xguide = "transformed read depth", yguide = "density", xlims = (0,1))
simulated_threeprime_termini = sample_from_kde(threeprime_termini.data, N)
plot!(kde(simulated_threeprime_termini); color = :tomato, label = "simulated 3' termini")

In [ ]:
function make_footprint_in_window(i::Int, fiveprime::Bool, threeprime::Bool)
    footprintlength = simulated_lengths[i]
    footprintstart = floor(Int, (window - footprintlength)/2)
    footprint = UnitRange(footprintstart, footprintstart + footprintlength - 1)
    padding = rand(2:5)

    #conservation
    footprint_conservation = simulated_conservation_peaks[i]
    conservation = fill(footprint_conservation, window)
    background = footprint_conservation * simulated_conservation_flanks[i]
    conservation[1:footprint.start-1+padding] .= background
    conservation[footprint.stop+1-padding:window] .= background

    #sRNA
    peak_height = simulated_peaks_srna[i]
    srna = fill(peak_height, window)
    background = peak_height * simulated_descents_srna[i]
    srna[1:footprint.start-1] .= background
    srna[footprint.stop+1:window] .= background

    #termini
    termini = zeros(Float64, window)
    if fiveprime
        termini[footprint.start] = simulated_fiveprime_termini[i]
    elseif threeprime
        termini[footprint.stop] = simulated_threeprime_termini[i]
    end
    hcat(conservation, srna, termini)
end


In [ ]:
N = 10000
if ~isdir("fiveprime_footprints")
    mkdir("fiveprime_footprints")
end
for n in 1:N
    footprint = make_footprint_in_window(n, true, false)
    serialize("fiveprime_footprints/$(string(n)).bin", footprint)
end

if ~isdir("threeprime_footprints")
    mkdir("threeprime_footprints")
end
for n in 1:N
    footprint = make_footprint_in_window(n, false, true)
    serialize("threeprime_footprints/$(string(n)).bin", footprint)
end

In [ ]:
#background

function removefeatures!(fwd_data, rev_data, features)
    for f in eachrow(features)
        if f.strand == "+"
            fwd_data[f.start:f.stop] .= -1
        else
            rev_data[rc(f.stop):rc(f.start)] .= -1
        end
    end
    vcat(fwd_data[fwd_data .>= 0], rev_data[rev_data .>= 0])
end

function shuffle(data::CircularVector{T}, N::Int) where T <: Real
    breakpoints = unique(data)
    links = Dict{T, Vector{Int}}()
    for bp in breakpoints
        links[bp] = findall(==(bp), data) .+ 1
    end
    samples = Vector{Vector{T}}(undef, N)
    for sample in 1:N
        idx = rand(1:length(data))
        window = Vector{T}(undef, 50)
        for i in eachindex(window)
            d = data[idx]
            window[i] = d
            idx = rand(links[d])
        end
        samples[sample] = window
    end
    samples
end

In [ ]:
foreground = vcat(structuralRNA, footprints_gff)
bg_conservation = removefeatures!(fwd_conservation, rev_conservation, foreground)
bg_srna = removefeatures!(fwd_srna, rev_srna, foreground)
bg_fiveprime = removefeatures!(fwd_fiveprime, rev_fiveprime, foreground)
bg_threeprime = removefeatures!(fwd_threeprime, rev_threeprime, foreground)

In [ ]:
N = 90000
background_conservation_samples = shuffle(bg_conservation, N)
background_srna_samples = shuffle(bg_srna, N)
background_fiveprime_samples = shuffle(bg_fiveprime, N)
background_threeprime_samples = shuffle(bg_threeprime, N)

if ~isdir("fiveprime_backgrounds")
    mkdir("fiveprime_backgrounds")
end
for n in 1:N
    footprint = hcat(background_conservation_samples[n], background_srna_samples[n], background_fiveprime_samples[n])
    serialize("fiveprime_backgrounds/$(string(n)).bin", footprint)
end

if ~isdir("threeprime_backgrounds")
    mkdir("threeprime_backgrounds")
end
for n in 1:N
    footprint = hcat(background_conservation_samples[n], background_srna_samples[n], background_threeprime_samples[n])
    serialize("threeprime_backgrounds/$(string(n)).bin", footprint)
end